In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

 78%|███████▊  | 53997/69120 [00:05<00:01, 13970.87it/s]

Error parsing /home/dzc/code/work/goose/experiments/_log_plan/pfcalcul/spanner_wl_none_equivalent-weighted_svr_plan_fd_6_0_18_0.log: 'NoneType' object has no attribute 'group'
Error parsing /home/dzc/code/work/goose/experiments/_log_plan/pfcalcul/spanner_wl_none_equivalent-weighted_rank-svm_plan_fd_2_0_09_0.log: 'NoneType' object has no attribute 'group'


100%|██████████| 69120/69120 [00:06<00:00, 10133.36it/s]


In [3]:
COLUMNS = [
    "facts",
    "optimiser",
    "features",
    "iterations",
    "feature_pruning",
    # "data_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    # df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "none"]
    # df = df[df.feature_pruning == "none"]
    return df

for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    # df = df.fillna(0)
    # replace 0 with NaN
    df = df.replace(0, -1)
    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)

solved


domain                                                blocksworld  childsnack  \
facts optimiser features iterations feature_pruning                             
all   rank-svm  wl       2          collapse-all               24          18   
                                    collapse-layer             22          10   
                                    collapse-layer-x           16          18   
                                    none                       23           9   
                         4          collapse-all               22          13   
                                    collapse-layer             22          11   
                                    collapse-layer-x           17          17   
                                    none                       21           9   
                         6          collapse-all               22          12   
                                    collapse-layer             23          11   
                                    collapse-layer-x           16          19   
                                    none                       -1           6   
      svr       wl       2          collapse-all                9           2   
                                    collapse-layer             11           2   
                                    collapse-layer-x           16           2   
                                    none                       10           2   
                         4          collapse-all               20           2   
                                    collapse-layer             12           2   
                                    collapse-layer-x           16           2   
                                    none                       20           2   
                         6          collapse-all               21           2   
                                    collapse-layer             10           2   
                                    collapse-layer-x           15           2   
                                    none                       21           2   
fd    rank-svm  wl       2          collapse-all               24           7   
                                    collapse-layer             24          11   
                                    collapse-layer-x           16          11   
                                    none                       23           7   
                         4          collapse-all               23          12   
                                    collapse-layer             23          14   
                                    collapse-layer-x           16          10   
                                    none                       21           9   
                         6          collapse-all               24          12   
                                    collapse-layer             23          11   
                                    collapse-layer-x           15          11   
                                    none                       -1          11   
      svr       wl       2          collapse-all               12           6   
                                    collapse-layer             11           6   
                                    collapse-layer-x           16           5   
                                    none                       11           6   
                         4          collapse-all               19           7   
                                    collapse-layer             10           6   
                                    collapse-layer-x           16           5   
                                    none                       21           7   
                         6          collapse-all               18           4   
                                    collapse-layer             12           4   
                                    collapse-layer-x           16           5   
                               

In [4]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        # print(len(df))
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="facts",
            line_dash="facts",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)